<a href="https://colab.research.google.com/github/newlightpooh/colab/blob/main/function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Library

In [2]:
import pandas as pd
import csv
import numpy as np
import math
import os
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from sklearn import metrics

In [4]:
# connect to google drive
import glob
from google.colab import drive
drive.mount('/content/drive')

# current path
print(os.getcwd())

# show directory
# print(os.listdir())

# show directory in each path
# data_path = 'drive/My Drive/'
# print(os.listdir(data_path))

# filelist = os.listdir(data_path)
# for filename in filelist:
#   print(filename)

Mounted at /content/drive
/content


#define Function

In [ ]:
parameter_split_dict = dict()
parameter_split_list = list()

fmt_parameter_dict = dict()

fmt_data_type_dict = dict()
#dict for type in pandas
data_types = {
    'a': 'int',
    'b': 'object', #int เพราะข้อมูลเหมือนเป็น id
    'B': 'object', #int เพราะข้อมูลเหมือนเป็น id
    'h': 'int',
    'H': 'int',
    'i': 'int',
    'I': 'int',
    'f': 'float',
    'd': 'float',
    'n': 'object',
    'N': 'object',
    'Z': 'object',
    'c': 'float',  #int
    'C': 'float',  #int
    'e': 'float',  #int
    'E': 'float',  #int
    'L': 'float',  #int
    'M': 'object',  #int
    'q': 'int',
    'Q': 'float'
}

parameter_from_sensor_list = [] #ชื่อ filter_parameter_split_list
parameter_data_list = []
header_list = []
non_filter_parameter_split_list = []
parameter_from_sensor_dict = dict()

all_subheader_list=[]

energy_consumption_time = pd.DataFrame()
equal_or_more_than_10_hz_parameter = list()
less_than_10_hz_parameter = list()

less_than_10_hz_order = []
equal_or_more_than_10_hz_order = []

parameter_same_freq_list = list()

result_df = pd.DataFrame()

filter_df = pd.DataFrame()

In [ ]:
def delete_all_file():
  directory_parameter = "drive/My Drive/Senior Project/parameter_collect"

  for filename in os.listdir(directory_parameter) :
      file_path = os.path.join(directory_parameter, filename)
      if os.path.isfile(file_path) :
        os.remove(file_path)

In [ ]:
def create_subdata_and_dict(filename):
  df = pd.read_csv('drive/My Drive/Senior Project/data/' + filename +'.log' , delimiter=',', names=range(30))
  df.iloc[:,1] = df.iloc[:,1] / 1000000

  parameter_split_dict = {name: i for i, name in enumerate(df.iloc[:, 0].unique())}

  df['group'] = df.iloc[:, 0].map(parameter_split_dict)

  # parameter_split_list = [group for _, group in df.groupby('group', sort=False)]
  parameter_split_list = [group.drop('group', axis=1).dropna(axis=1, how='all').applymap(lambda x: x.strip() if isinstance(x, str) else x) for _, group in df.groupby('group', sort=False)]  # drop column group ที่สร้างไว้ใช้ตอน groupby และ drop column ที่ทุก row เป็น nan

  return parameter_split_list,parameter_split_dict

In [ ]:
def fmt_to_csv():
  parameter_split_list[0].to_csv('drive/My Drive/Senior Project/parameter_collect/{}.csv'.format(parameter_split_list[0].iloc[0, 0]), index=False, header=False)

  # get parameter with sub parameter
  parameter_fmt = open('drive/My Drive/Senior Project/parameter_collect/FMT.csv')
  fmt_reader = csv.reader(parameter_fmt)

  fmt_rows = list()

  for row in fmt_reader :
      del row[0:3]
      del row[1]
      row = [value.strip() for value in row if value != '']
      fmt_rows.append(row)

  for i in range(len(fmt_rows)) :
      for j in range(2, len(fmt_rows[i])) :
          fmt_rows[i][j] = fmt_rows[i][0] + '_' + fmt_rows[i][j]

  # create dict to collect parameter with sub parameter
  for i in range(len(fmt_rows)) :
      if len(fmt_rows[i]) > 0:
          fmt_parameter_dict[fmt_rows[i][0]] = fmt_rows[i]

  return fmt_parameter_dict

In [ ]:
def assign_type_header_to_subdata():
  parameter_fmt = open('drive/My Drive/Senior Project/parameter_collect/FMT.csv')
  fmt_reader = csv.reader(parameter_fmt)

  fmt_rows = list()

  for row in fmt_reader :
      row_type = list()
      row_type.append(row[3].strip())
      row_type.extend([value for value in row[4].strip()])
      fmt_rows.append(row_type)

  for i in range(len(fmt_rows)) :
      #ไว้สำหรับชื่อ row group parameter
      modified_row = fmt_rows[i].copy()
      modified_row[0] = "Z"
      modified_row = [data_types.get(item, item) for item in modified_row]
      fmt_data_type_dict[fmt_rows[i][0]] = modified_row

  #ให้ type ใน df ตาม fmt_data_type_dict
  #ให้ header ตาม fmt_parameter_dict
  for df in parameter_split_list[1:]:
    # print(df.iloc[0, 0])
    df.columns = fmt_parameter_dict[df.iloc[0, 0]]
    data_type = fmt_data_type_dict[df.iloc[0, 0]]
    for col, dtype in zip(df.columns, data_type):
      df[col] = df[col].astype(dtype)

  return fmt_data_type_dict

In [ ]:
def subdata_to_csv():
  for i in range(1,len(parameter_split_list)):
    filename = 'drive/My Drive/Senior Project/parameter_collect/{}.csv'.format(parameter_split_list[i].iloc[0, 0])

    header_list = fmt_parameter_dict[parameter_split_list[i].iloc[0, 0]]

    df_with_header = pd.DataFrame(parameter_split_list[i].values, columns=header_list) #drop column ที่มี nan

    df_with_header.to_csv(filename, index=False)

In [ ]:
def filter_and_split_subInsub_and_create_subdata_and_dict():
  for i in range(len(parameter_split_list)):
    df = parameter_split_list[i]
    # if type(df.columns[2]) != int: #ไม่เอาFMT
    if i != parameter_split_dict['FMT'] :
      if (((len(df.columns[2].split('_')[1]) == 1) or (df.columns[2].split('_')[1] in ['chan', 'IMU'])) and df.dtypes[2] == object):
        temp_df = df.copy()
        # temp_df.iloc[:, 2] = temp_df.iloc[:, 2].astype(int)
        temp_df[temp_df.columns[2]] = temp_df[temp_df.columns[2]].astype(int)
        grouped_df = temp_df.groupby(temp_df.columns[2])
        name = temp_df.columns[0]
        num_groups = temp_df[temp_df.columns[2]].nunique()
        if num_groups>1:
          for group_name, group_data in grouped_df:
            column_name_mapping = {col: col.replace(name, name+"["+str(group_name)+"]") for col in group_data.columns if name in col}
            group_data.rename(columns=column_name_mapping, inplace=True)
            # print(name+"_"+str(group_name))
            group_data.iloc[:, 2] = group_data.iloc[:, 2].astype(object)
            non_filter_parameter_split_list.append(group_data)
          # print(name)
        else:
          non_filter_parameter_split_list.append(df)
      else:
        non_filter_parameter_split_list.append(df)


  for df in non_filter_parameter_split_list:
      df_before = df.iloc[:, :2]
      df_after = df.iloc[:, 2:]
      df_after = df_after.select_dtypes(exclude='object')

      if len(df_after.columns) > 0:
        df_result = pd.concat([df_before, df_after], axis=1)
        parameter_from_sensor_list.append(df_result.columns[0])
        header_list.append(df_result.columns.tolist())
        parameter_data_list.append(df_result)

  parameter_from_sensor_dict = {param: idx for idx, param in enumerate(parameter_from_sensor_list)}

  return parameter_from_sensor_dict


In [ ]:
def create_all_header():
  all_subheader_list.append('TimeUS')

  for i in range(len(header_list)) :
      for j in range(2, len(header_list[i])) :
          all_subheader_list.append(header_list[i][j])

In [ ]:
def find_less_or_more_freq():
  energy_consumption_time = parameter_data_list[parameter_from_sensor_dict['BAT']].iloc[:,1].astype(float)

  time_diff_energy = energy_consumption_time.shift(-1) - energy_consumption_time
  avg_time_diff_energy = time_diff_energy.dropna().mean()
  # print(avg_time_diff_energy)

  for i in range(len(parameter_data_list)) :
      # if float(parameter_data_list[i][2][1]) - float(parameter_data_list[i][1][1]) > time_diff + 0.02 and float(parameter_data_list[i][2][1]) - float(parameter_data_list[i][1][1]) > time_diff - 0.02 :
      time_parameter = parameter_data_list[i].iloc[:,1].astype(float)
      time_diff_parameter = time_parameter.shift(-1) - time_parameter
      avg_time_diff_parameter = time_diff_parameter.dropna().mean()
      # print(avg_time_diff_parameter,parameter_data_list[i].iloc[0,0])
      if  avg_time_diff_parameter > avg_time_diff_energy  :
          # less_than_10_hz_parameter.append(parameter_data_list[i].iloc[0,0])
          less_than_10_hz_parameter.append(parameter_data_list[i].columns[0])
      else :
          # equal_or_more_than_10_hz_parameter.append(parameter_data_list[i].iloc[0,0])
          equal_or_more_than_10_hz_parameter.append(parameter_data_list[i].columns[0])

  return energy_consumption_time

In [ ]:
def index_in_less_or_more_list():
  lt_count = 0
  eom_count = 0
  for i in range(len(parameter_data_list)) :
      if lt_count < len(less_than_10_hz_parameter) and parameter_data_list[i].columns[0] == less_than_10_hz_parameter[lt_count] :
          less_than_10_hz_order.append(i)
          lt_count += 1
      elif eom_count < len(equal_or_more_than_10_hz_parameter)  and parameter_data_list[i].columns[0] == equal_or_more_than_10_hz_parameter[eom_count]:
          equal_or_more_than_10_hz_order.append(i)
          eom_count += 1

In [ ]:
def undersampling():
  for k in equal_or_more_than_10_hz_order:
      df_low_freq = pd.DataFrame(energy_consumption_time.values, columns=['TimeUS'])

      df_high_freq = parameter_data_list[k]

      merged_df = pd.merge_asof(df_low_freq, df_high_freq, on='TimeUS', direction='nearest')

      merged_df.drop(merged_df.columns[[0, 1]],axis = 1 ,inplace = True)

      parameter_same_freq_list.append(merged_df)

In [ ]:
def oversampling():
  for k in less_than_10_hz_order :

    df_high_freq = pd.DataFrame(energy_consumption_time.values, columns=['TimeUS'])
    df_high_freq['index'] = df_high_freq.index

    df_low_freq = pd.DataFrame(parameter_data_list[k])
    # parameter_name = parameter_data_list[k].iloc[0,0]
    original_types = df_low_freq.iloc[:, 1:].dtypes

    merged_df = pd.merge_asof(df_low_freq, df_high_freq, on='TimeUS', direction='nearest')

    merged_df.loc[merged_df.duplicated(subset=['index'], keep='last'), 'index'] = pd.NA
    merged_df.drop('TimeUS', axis=1, inplace=True)
    merged_df = pd.merge(df_high_freq, merged_df, on='index', how='left')
    merged_df.drop('index',axis = 1 ,inplace = True)

    merged_df.iloc[:, 1:] = merged_df.iloc[:, 1:].apply(lambda col: col.interpolate(method='linear',limit_direction='both'), axis=0)
    merged_df = merged_df.astype(original_types)
    merged_df.drop(merged_df.columns[[0, 1]],axis = 1 ,inplace = True)
    # merged_df[parameter_name].fillna(parameter_name, inplace=True)  #fill ชื่อ เพราะ nan จาก ไม่มี index ที่ตรง

    parameter_same_freq_list.append(merged_df)
    # t2_parameter_same_freq_list.append(merged_df)


In [ ]:
def complete_data():
  result_df  =  pd.concat(parameter_same_freq_list,axis = 1)
  result_df = pd.concat([energy_consumption_time.reset_index(drop=True),result_df],axis = 1)

  new_data = pd.DataFrame({'BAT_VoltCurr': result_df['BAT_Volt'] * result_df['BAT_Curr'],
                        'BAT_EnrgDiff': result_df['BAT_EnrgTot'] - result_df['BAT_EnrgTot'].shift(1)},index=list(range(len(result_df))))

  result_df = pd.concat([result_df, new_data], axis=1 )

  result_df["BAT_EnrgDiff"].fillna(0, inplace=True)
  return result_df
  # result_df.to_csv(dir_name+global_filename +'.csv', index=False)

In [ ]:
def filter_mode():
  mode_df = parameter_split_list[parameter_split_dict['MODE']].iloc[:,1:3]
  merged_df = pd.merge_asof(result_df, mode_df, on='TimeUS', direction='backward')
  merged_df['MODE_Mode'] = merged_df['MODE_Mode'].bfill()
  merged_df['MODE_Mode'] = merged_df['MODE_Mode'].ffill()
  filter_df = merged_df[merged_df['MODE_Mode'].isin(['FBWA', 'Auto'])].drop('MODE_Mode', axis=1)
  return filter_df

In [ ]:
# result_df.to_csv(dir_name+global_filename +'.csv', index=False)

In [ ]:
def main(filename):
  global parameter_split_dict, parameter_split_list, fmt_parameter_dict, fmt_data_type_dict
  global parameter_from_sensor_list, parameter_data_list, header_list, non_filter_parameter_split_list
  global parameter_from_sensor_dict, all_subheader_list, energy_consumption_time
  global equal_or_more_than_10_hz_parameter, less_than_10_hz_parameter
  global less_than_10_hz_order, equal_or_more_than_10_hz_order, parameter_same_freq_list
  global result_df, filter_df

  delete_all_file()
  parameter_split_list,parameter_split_dict = create_subdata_and_dict(filename)
  fmt_parameter_dict = fmt_to_csv()
  fmt_data_type_dict = assign_type_header_to_subdata()
  # subdata_to_csv()
  parameter_from_sensor_dict = filter_and_split_subInsub_and_create_subdata_and_dict()
  create_all_header()
  energy_consumption_time = find_less_or_more_freq()
  index_in_less_or_more_list()
  undersampling()
  oversampling()
  result_df = complete_data()
  filter_df = filter_mode()
  return result_df

# main

In [ ]:
files = glob.glob(f"drive/My Drive/Senior Project/data/*.log")
dir_name = 'drive/My Drive/Senior Project/Complete_combine/2_to_filter_mode/'


for file in files:
    parameter_split_dict = dict()
    parameter_split_list = list()

    fmt_parameter_dict = dict()

    fmt_data_type_dict = dict()
    #dict for type in pandas
    data_types = {
        'a': 'int',
        'b': 'object', #int เพราะข้อมูลเหมือนเป็น id
        'B': 'object', #int เพราะข้อมูลเหมือนเป็น id
        'h': 'int',
        'H': 'int',
        'i': 'int',
        'I': 'int',
        'f': 'float',
        'd': 'float',
        'n': 'object',
        'N': 'object',
        'Z': 'object',
        'c': 'float',  #int
        'C': 'float',  #int
        'e': 'float',  #int
        'E': 'float',  #int
        'L': 'float',  #int
        'M': 'object',  #int
        'q': 'int',
        'Q': 'float'
    }

    parameter_from_sensor_list = [] #ชื่อ filter_parameter_split_list
    parameter_data_list = []
    header_list = []
    non_filter_parameter_split_list = []
    parameter_from_sensor_dict = dict()

    all_subheader_list=[]

    energy_consumption_time = pd.DataFrame()
    equal_or_more_than_10_hz_parameter = list()
    less_than_10_hz_parameter = list()

    less_than_10_hz_order = []
    equal_or_more_than_10_hz_order = []

    parameter_same_freq_list = list()

    result_df = pd.DataFrame()

    filter_df = pd.DataFrame()

    filename, _ = os.path.splitext(os.path.basename(file))
    filter_df = main(filename)
    filter_df.to_csv(dir_name+filename +'.csv', index=False)

<ipython-input-6-537002ec3680>:2: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('drive/My Drive/Senior Project/data/' + filename +'.log' , delimiter=',', names=range(30))
<ipython-input-6-537002ec3680>:2: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('drive/My Drive/Senior Project/data/' + filename +'.log' , delimiter=',', names=range(30))
<ipython-input-6-537002ec3680>:2: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('drive/My Drive/Senior Project/data/' + filename +'.log' , delimiter=',', names=range(30))
<ipython-input-6-537002ec3680>:2: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20) have mixed types. Sp

In [14]:
csv_file = 'drive/My Drive/Senior Project/definition.csv'

with open(csv_file, 'r') as file:
    reader = csv.DictReader(file)

    data_dict = [row for row in reader]
    data_dict = data_dict[0]

In [22]:
files = glob.glob(f"drive/My Drive/Senior Project/Complete_combine/2_to_filter_mode/*.csv")

list_df = []
list_header = []

for file in files:
    filename, _ = os.path.splitext(os.path.basename(file))
    # print(filename)
    all_parameter_df = pd.read_csv(file)
    list_header.append(filename)
    all_parameter_df = all_parameter_df.drop(['TimeUS','BAT_EnrgDiff'],errors='ignore', axis = 1)

    df = all_parameter_df
    df.fillna(0, inplace=True)
    df = df.loc[:, (df != 0).any(axis=0)] #ลบทุก column ที่ทุก row = 0
    parameter_list = df.columns

    df_corr = df.copy()
    correlation_all_parameter = df_corr.corr()

    correlation_with_powr = correlation_all_parameter.loc[correlation_all_parameter.index != 'BAT_VoltCurr', 'BAT_VoltCurr'].to_frame()  #ในข้อมูลพี่ริว คือ 21
    correlation_with_powr['Abs_Corr'] = correlation_with_powr["BAT_VoltCurr"].abs()

    sorted_correlation_with_powr = correlation_with_powr.sort_values(by='Abs_Corr', ascending=False).copy()
    sorted_correlation_with_powr.drop('Abs_Corr',axis = 1 ,inplace = True)
    sorted_correlation_with_rank = sorted_correlation_with_powr.iloc[:100].copy()
    sorted_correlation_with_rank.loc[:, 'rank'] = range(1, 101)
    sorted_correlation_with_rank.drop('BAT_VoltCurr',axis = 1 ,inplace = True)

    list_df.append(sorted_correlation_with_rank)


In [9]:
filename = "drive/My Drive/Senior Project/Complete_combine/filter_mode_to_phase/no_climb1.csv"
df = pd.read_csv(filename,index_col=0)

In [10]:
df

,Unnamed: 0,IMU_GyrX,IMU_GyrY,IMU_GyrZ,IMU_AccX,IMU_AccY,IMU_AccZ,IMU_EG,IMU_EA,IMU_T,...,PM_Ex,QTUN_Sscl,PSCD_TPD,PSCD_PD,PSCD_DVD,PSCD_TVD,PSCD_VD,PSCD_DAD,PSCD_TAD,PSCD_AD
0,765,-0.636056,-0.296320,0.083804,0.028076,0.315510,-6.574552,0,0,50.89222,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,766,-0.372304,-0.293935,0.060367,-0.687005,0.509251,-5.650390,0,0,50.88937,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,767,-0.192166,0.046339,0.021832,-0.282436,0.394145,-5.125965,0,0,50.88906,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,768,-0.082395,0.211138,-0.012195,-0.259242,0.400299,-8.571963,0,0,50.89001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,769,-0.011564,0.113162,-0.035860,-0.552502,0.649396,-10.699450,0,0,50.89030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47448,6412,0.013701,-0.042942,0.007541,0.336339,0.649586,-9.799566,0,0,44.87757,...,0.0,0.0,0.410268,0.400516,0.0,0.009752,0.030923,0.0,-0.113173,-0.036573
47449,6413,0.008215,-0.049347,0.005170,1.062206,1.100587,-10.154850,0,0,44.87419,...,0.0,0.0,0.410268,0.402001,0.0,0.008267,0.040251,0.0,-0.126234,0.220550
47450,6414,0.001314,-0.048434,0.003576,0.205274,0.811976,-9.765584,0,0,44.87576,...,0.0,0.0,0.410268,0.404218,0.0,0.006050,0.032661,0.0,-0.142700,0.347426
47451,6415,-0.000763,-0.060557,0.002839,0.062731,1.226167,-10.067140,0,0,44.88554,...,0.0,0.0,0.410268,0.405078,0.0,0.005190,0.022764,0.0,-0.137060,-0.282687


In [17]:
list_df = []
list_header = []

filename = "drive/My Drive/Senior Project/Complete_combine/filter_mode_to_phase/no_climb+turn_phase1.csv"

# print(filename)
all_parameter_df = pd.read_csv(filename,index_col=0)
list_header.append(filename)
all_parameter_df = all_parameter_df.drop(['TimeUS','BAT_EnrgDiff'],errors='ignore', axis = 1)

df = all_parameter_df
df.fillna(0, inplace=True)
df = df.loc[:, (df != 0).any(axis=0)] #ลบทุก column ที่ทุก row = 0
parameter_list = df.columns

df_corr = df.copy()
correlation_all_parameter = df_corr.corr()

correlation_with_powr = correlation_all_parameter.loc[correlation_all_parameter.index != 'BAT_VoltCurr', 'BAT_VoltCurr'].to_frame()  #ในข้อมูลพี่ริว คือ 21
correlation_with_powr['Abs_Corr'] = correlation_with_powr["BAT_VoltCurr"].abs()

sorted_correlation_with_powr = correlation_with_powr.sort_values(by='Abs_Corr', ascending=False).copy()
sorted_correlation_with_powr.drop('Abs_Corr',axis = 1 ,inplace = True)
sorted_correlation_with_rank = sorted_correlation_with_powr.iloc[:100].copy()
sorted_correlation_with_rank.loc[:, 'rank'] = range(1, 101)
sorted_correlation_with_rank.drop('BAT_VoltCurr',axis = 1 ,inplace = True)

list_df.append(sorted_correlation_with_rank)

In [18]:
filepath = 'drive/My Drive/Senior Project/rank_corr_no_climb+turn_phase.csv'

result_df = pd.concat(list_df,axis = 1)
result_df['index'] = result_df.index.str.replace(r'\[.*?\]', '')
result_df.insert(0, 'definition', result_df['index'].map(data_dict))
result_df.drop('index',axis = 1 ,inplace = True)
result_df.to_csv(filepath, index=True)

<ipython-input-18-61075d21b993>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  result_df['index'] = result_df.index.str.replace(r'\[.*?\]', '')


In [16]:
result_df

,definition,rank
BAT_Curr,measured current,1
RCOU_C14,channel 14 output,2
RCOU_C13,channel 13 output,3
RCOU_C9,channel 9 output,4
AETR_Thr,Pre-mixer value for throttle output (between -...,5
...,...,...
PSCE_TPE,Target position relative to EKF origin,96
PIQP_P,proportional part of PID,97
PIDP_P,proportional part of PID,98
XKY0_W0,Weighting applied to yaw estimate from individ...,99


In [7]:
result_df = pd.read_csv('drive/My Drive/Senior Project/rank_corr_all_flight2.csv')
result_df

,Unnamed: 0,definition,00000021,00000132,00000152,00000153,00000154,00000155
0,BAT_Curr,measured current,1.0,1.0,1.0,1.0,1.0,1.0
1,RCOU_C9,channel 9 output,2.0,2.0,6.0,2.0,3.0,4.0
2,RCOU_C13,channel 13 output,3.0,3.0,7.0,3.0,5.0,3.0
3,RCOU_C14,channel 14 output,4.0,4.0,8.0,4.0,4.0,2.0
4,IMU3_AccX,NaN,5.0,7.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
322,XKF1[0]_PE,NaN,NaN,NaN,NaN,NaN,NaN,92.0
323,AHR2_Lng,Estimated longitude,NaN,NaN,NaN,NaN,NaN,93.0
324,GPS_Lng,longitude,NaN,NaN,NaN,NaN,NaN,94.0
325,XKF3[1]_IMZ,NaN,NaN,NaN,NaN,NaN,NaN,95.0


In [20]:
result_df[result_df.iloc[:, 0]=='NKF1[0]_PE']

,Unnamed: 0,definition,00000021,00000132,00000152,00000153,00000154,00000155


In [21]:
df_no_missing_values = result_df.dropna(subset=result_df.columns[-6:])
df_no_missing_values

,Unnamed: 0,definition,00000021,00000132,00000152,00000153,00000154,00000155
0,BAT_Curr,measured current,1.0,1.0,1.0,1.0,1.0,1.0
1,RCOU_C9,channel 9 output,2.0,2.0,6.0,2.0,3.0,4.0
2,RCOU_C13,channel 13 output,3.0,3.0,7.0,3.0,5.0,3.0
3,RCOU_C14,channel 14 output,4.0,4.0,8.0,4.0,4.0,2.0
5,AETR_Thr,Pre-mixer value for throttle output (between -...,6.0,5.0,10.0,5.0,7.0,5.0
19,NTUN_AspdE,NaN,20.0,62.0,90.0,70.0,65.0,19.0


In [5]:
filename = "drive/My Drive/Senior Project/Complete_combine/filter_mode_to_phase/no_climb.csv"
direct_phase = pd.DataFrame()
turn_phase = pd.DataFrame()

df = pd.read_csv(filename,index_col=0)
direct_phase = df[df['ATT_Roll'] <= 10]
turn_phase = df[df['ATT_Roll'] > 10]
# direct_phase.to_csv("drive/My Drive/Senior Project/Complete_combine/filter_mode_to_phase/direct_phase.csv", index=True)
# turn_phase.to_csv("drive/My Drive/Senior Project/Complete_combine/filter_mode_to_phase/turn_phase.csv", index=True)

In [6]:
X = df.drop(['TimeUS','BAT_EnrgDiff'],errors='ignore', axis = 1)
X.fillna(0, inplace=True)
X = X.loc[:, (X != 0).any(axis=0)]
Y = X['BAT_VoltCurr']
X = X.drop('BAT_VoltCurr', axis=1)

In [7]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
temp = X.values
sc = StandardScaler()
X_norm = sc.fit_transform(temp)

In [8]:
pca = PCA(n_components=2)

# Fit and transform your data
X_pca = pca.fit_transform(X_norm)
explained_variance_ratio = pca.explained_variance_ratio_
loadings = pca.components_

loadings_df = pd.DataFrame(loadings, columns=X.columns)
sorted_loadings = loadings_df.T
# sorted_loadings_pc1 = loadings_df.T['1'].sort_values(ascending=False)

sorted_loadings.loc[:,0].sort_values(ascending=False)

NKT3_EKFMax    0.076162
NKT2_EKFMax    0.076155
NKT1_EKFMax    0.076139
NKT3_EKFMin    0.076126
XKF7_MN        0.076123
                 ...   
RCI2_C16      -0.075999
RCI2_C15      -0.075999
XKT_EKFMin    -0.075999
XKT_EKFMax    -0.076002
CTUN_E2T      -0.076011
Name: 0, Length: 586, dtype: float64

In [ ]:
# turn_phase.to_csv("drive/My Drive/Senior Project/Complete_combine/filter_mode_to_phase/turn_phase.csv", index=True)